In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
import scipy

from scipy.io import mmwrite, mmread
from scipy.sparse import csr_matrix

import sctk

In [2]:
import session_info
session_info.show()

In [3]:
sc.settings.set_figure_params(dpi=120)

# Variables

In [4]:
cells_to_analyse = ['VentricularCardiomyocytesLeftCompact',
                     'VentricularCardiomyocytesLeftTrabeculated',
                    'VentricularCardiomyocytesRightCompact',
                     'VentricularCardiomyocytesRightTrabeculated',
                    'VentricularCardiomyocytesCycling']

celltype_key = 'fine_grain'

# region_to_analyse = ['whole sample']

# Read in

In [5]:
!ls /nfs/team205/heart/anndata_objects/Foetal

14Feb_fromSemih		      QC
archive			      RNA_foetal_post-scVI_raw.h5ad
EC			      RNA_foetal_raw.h5ad
Feb28ObjectRaw.h5ad	      scArches
Feb28ObjectRaw_Multiome.h5ad  scVI
Feb28ObjectRaw_Multiome.rds   TipAndStalkAndGVSMCsPartitionedForCell2Loc.h5ad
Feb28ObjectRaw.obs.csv	      trisomy21
Feb28ObjectRaw_original.h5ad  visium-FFPE_foetal_raw.h5ad
Feb28ObjectRaw_re-scVI.h5ad   visium_foetal_healthy-all_raw.h5ad
FetalAtlas.h5ad		      visium_foetal_healthy-all_spot-annotation.csv
He_Cell_foetal-lung	      visium_foetal_raw.h5ad
Hill2022.h5ad		      visium-OCT_foetal_acrossage_raw.h5ad
Hill2022_QCapplied.h5ad       visium-OCT_foetal_DuctusA_raw.h5ad
Immune			      visium-OCT_foetal_raw.h5ad
metadata		      visium-OCT-LV_foetal_raw.h5ad
multiome_ATAC		      visium-OCT-T21_foetal_raw.h5ad
multiVI


In [6]:
# rna
adata_rna = sc.read_h5ad('/nfs/team205/heart/anndata_objects/Foetal/Feb28ObjectRaw.h5ad')
print(adata_rna.obs[celltype_key].value_counts())

# subset cells
adata_rna = adata_rna[adata_rna.obs[celltype_key].isin(cells_to_analyse)]

# subset regions
# adata_rna = adata_rna[adata_rna.obs['region'].isin(region_to_analyse)]

# add modality to .var
adata_rna.var['modality']='Gene Expression'

adata_rna

GreatVesselAdventitialFibroblasts        61593
GreatVesselSmoothMuscleCells             22992
VentricularCardiomyocytesRightCompact    16215
VentricularCardiomyocytesLeftCompact     15591
MyocardialInterstitialFibroblasts1       13880
                                         ...  
MacrophagesTIMD4pos                        470
MonocytesMPOpos                            414
Megakaryocytes                             360
ChromaffinCells                            228
DendriticCellsMature                       184
Name: fine_grain, Length: 63, dtype: int64


AnnData object with n_obs × n_vars = 50099 × 36601
    obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'sangerID', 'combinedID', 'donor', 'region', 'age', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'scrublet_score', 'doublet_pval', 'doublet_bh_pval', 'n_genes', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'HB_score', 'donor_by_library-prep', 'multiplexed', 'SOC | status', 'SOC | log_prob_singleton', 'SOC | log_prob_doublet', 'batch_key', '_scvi_batch', 'FACSgate', 'fine_grain', 'mid_grain', 'coarse_grain', 'sex', 'week', 'trimester', 'heart_or_greatvessels', 'cycling', 'S_score', 'G2M_score', 'phase', '_scvi_labels', 'stress_score', 'hb1_score'
    var: 'gene_name_scRNA-0', 'gene_id', 'modality'
    uns: 'FACSgate_colors', '_scvi_manager_uuid', '_scvi_uuid', 'age_colors', 'c

In [7]:
pd.crosstab(adata_rna.obs[celltype_key],adata_rna.obs['region'])

region,apex,atria,atrial septum,base,whole sample,heart without node,left ventricle,node,outflow tract,right ventricle,right ventricle and papillaries
fine_grain,,,,,,,,,,,
VentricularCardiomyocytesLeftCompact,1263,214,10,139,10692,1594,1029,0,46,299,305
VentricularCardiomyocytesRightCompact,663,399,7,342,8246,819,371,1,89,2475,2803
VentricularCardiomyocytesLeftTrabeculated,476,86,6,127,4720,720,701,8,50,60,43
VentricularCardiomyocytesRightTrabeculated,195,45,0,55,1698,260,40,0,13,293,502
VentricularCardiomyocytesCycling,1181,105,2,254,4957,563,210,3,85,369,466


In [8]:
pd.crosstab(adata_rna.obs[celltype_key],adata_rna.obs['kit_10x'])

kit_10x,3prime-v3,5prime-v2,Multiome-v1
fine_grain,,,
VentricularCardiomyocytesLeftCompact,1201,1214,13176
VentricularCardiomyocytesRightCompact,960,1484,13771
VentricularCardiomyocytesLeftTrabeculated,457,663,5877
VentricularCardiomyocytesRightTrabeculated,247,254,2600
VentricularCardiomyocytesCycling,1167,1173,5855


In [25]:
pd.crosstab(adata_rna.obs[celltype_key],adata_rna.obs['cell_or_nuclei'])

cell_or_nuclei,cell,nuclei
fine_grain,,
VentricularCardiomyocytesLeftCompact,2415,13176
VentricularCardiomyocytesRightCompact,2444,13771
VentricularCardiomyocytesLeftTrabeculated,1120,5877
VentricularCardiomyocytesRightTrabeculated,501,2600
VentricularCardiomyocytesCycling,2340,5855


In [26]:
pd.crosstab(adata_rna.obs['age'],adata_rna.obs['cell_or_nuclei'])

cell_or_nuclei,cell,nuclei
age,,
4W3D,0,946
5W4D,0,362
6W1D,942,0
6W4D,266,0
6W5D,772,0
7W0D,0,454
7W1D,947,0
7W3D,528,0
7W5D,1356,0


In [9]:
pd.crosstab(adata_rna.obs[celltype_key],adata_rna.obs['cycling'])

cycling,no,yes
fine_grain,,
VentricularCardiomyocytesLeftCompact,10825,4766
VentricularCardiomyocytesRightCompact,11322,4893
VentricularCardiomyocytesLeftTrabeculated,4694,2303
VentricularCardiomyocytesRightTrabeculated,2372,729
VentricularCardiomyocytesCycling,0,8195


In [10]:
adata_rna.obs['kit_10x'].value_counts()

Multiome-v1    41279
5prime-v2       4788
3prime-v3       4032
Name: kit_10x, dtype: int64

In [11]:
# atac
# ie only Multiome data
adata_atac = sc.read_h5ad('/nfs/team205/heart/anndata_objects/Foetal/multiome_ATAC/ArchR/project_output/PeakMatrix/Foetal_Peaks.h5ad')

# subset cells
adata_atac = adata_atac[adata_atac.obs[celltype_key].isin(cells_to_analyse)]
print(adata_atac.obs[celltype_key].value_counts())

# subset regions
# adata_atac = adata_atac[adata_atac.obs['region'].isin(region_to_analyse)]

# add modality to .var
adata_atac.var['modality']='Peaks'

adata_atac

VentricularCardiomyocytesRightCompact         12008
VentricularCardiomyocytesLeftCompact          11397
VentricularCardiomyocytesLeftTrabeculated      5054
VentricularCardiomyocytesCycling               4977
VentricularCardiomyocytesRightTrabeculated     2269
Name: fine_grain, dtype: int64


AnnData object with n_obs × n_vars = 35705 × 508040
    obs: 'sangerID', 'combinedID', 'donor', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'multiplexed', 'batch_key', 'region', 'FACSgate', 'sex', 'week', 'trimester', 'fine_grain', 'mid_grain', 'coarse_grain', 'heart_or_greatvessels'
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'distToGeneStart', 'nearestGene', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N', 'modality'

In [12]:
pd.crosstab(adata_atac.obs[celltype_key],adata_atac.obs['region'])

region,apex,atria,atrial septum,heart without node,left ventricle,node,outflow tract,right ventricle,right ventricle and papillaries,whole sample
fine_grain,,,,,,,,,,
VentricularCardiomyocytesCycling,185,101,1,477,180,3,51,355,370,3254
VentricularCardiomyocytesLeftCompact,168,207,9,1340,890,0,41,288,243,8211
VentricularCardiomyocytesLeftTrabeculated,122,82,5,589,608,6,44,55,28,3515
VentricularCardiomyocytesRightCompact,38,385,6,714,337,1,58,2372,2291,5806
VentricularCardiomyocytesRightTrabeculated,3,42,0,231,35,0,10,281,420,1247


In [13]:
adata_atac.obs

,sangerID,combinedID,donor,facility,cell_or_nuclei,modality,kit_10x,multiplexed,batch_key,region,FACSgate,sex,week,trimester,fine_grain,mid_grain,coarse_grain,heart_or_greatvessels
7089STDY13216921_BHF_F_Hea13242528_TGCATTAAGGACCTTG-1,7089STDY13216921,7089STDY13216921_BHF_F_Hea13242528,Hst40,Sanger,nuclei,scMultiome,Multiome-v1,Yes,Hst40_Nuclei_Multiome-v1,node,na,male,14W,secondtrimester,VentricularCardiomyocytesCycling,VentricularCardiomyocytes,Cardiomyocytes,heart
7089STDY13216921_BHF_F_Hea13242528_TGCTCACTCAGGCCTA-1,7089STDY13216921,7089STDY13216921_BHF_F_Hea13242528,Hst40,Sanger,nuclei,scMultiome,Multiome-v1,Yes,Hst40_Nuclei_Multiome-v1,node,na,male,14W,secondtrimester,VentricularCardiomyocytesLeftTrabeculated,VentricularCardiomyocytes,Cardiomyocytes,heart
7089STDY13216921_BHF_F_Hea13242528_ATGTCAATCCTCCCTC-1,7089STDY13216921,7089STDY13216921_BHF_F_Hea13242528,Hst40,Sanger,nuclei,scMultiome,Multiome-v1,Yes,Hst40_Nuclei_Multiome-v1,node,na,male,14W,secondtrimester,VentricularCardiomyocytesLeftTrabeculated,VentricularCardiomyocytes,Cardiomyocytes,heart
7089STDY13216921_BHF_F_Hea13242528_GGGCATGAGAATCTCA-1,7089STDY13216921,7089STDY13216921_BHF_F_Hea13242528,Hst40,Sanger,nuclei,scMultiome,Multiome-v1,Yes,Hst40_Nuclei_Multiome-v1,node,na,male,14W,secondtrimester,VentricularCardiomyocytesLeftTrabeculated,VentricularCardiomyocytes,Cardiomyocytes,heart
7089STDY13216921_BHF_F_Hea13242528_GGAACCTGTGGGTACT-1,7089STDY13216921,7089STDY13216921_BHF_F_Hea13242528,Hst40,Sanger,nuclei,scMultiome,Multiome-v1,Yes,Hst40_Nuclei_Multiome-v1,node,na,male,14W,secondtrimester,VentricularCardiomyocytesLeftTrabeculated,VentricularCardiomyocytes,Cardiomyocytes,heart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BHF_F_Hea11933675_BHF_F_Hea11596628_GCCTGCTGTCACGGAT-1,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,Sanger,nuclei,scMultiome,Multiome-v1,No,C98_Nuclei_Multiome-v1,whole sample,na,male,7W,firsttrimester,VentricularCardiomyocytesCycling,VentricularCardiomyocytes,Cardiomyocytes,heart
BHF_F_Hea11933675_BHF_F_Hea11596628_ATCTTTGGTCAAGTGC-1,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,Sanger,nuclei,scMultiome,Multiome-v1,No,C98_Nuclei_Multiome-v1,whole sample,na,male,7W,firsttrimester,VentricularCardiomyocytesRightCompact,VentricularCardiomyocytes,Cardiomyocytes,heart
BHF_F_Hea11933675_BHF_F_Hea11596628_ATCCATAAGTCGCAAT-1,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,Sanger,nuclei,scMultiome,Multiome-v1,No,C98_Nuclei_Multiome-v1,whole sample,na,male,7W,firsttrimester,VentricularCardiomyocytesRightCompact,VentricularCardiomyocytes,Cardiomyocytes,heart
BHF_F_Hea11933675_BHF_F_Hea11596628_CCGTTAACAGCCTTGG-1,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,Sanger,nuclei,scMultiome,Multiome-v1,No,C98_Nuclei_Multiome-v1,whole sample,na,male,7W,firsttrimester,VentricularCardiomyocytesCycling,VentricularCardiomyocytes,Cardiomyocytes,heart


# Prepare paired anndata

In [14]:
# shared barcodes, ie Multiome data (not all, since MultiomeRNA has a bit more data than MultiomeATAC)
barcodes_inter=list(set(adata_rna.obs_names).intersection(adata_atac.obs_names))
len(barcodes_inter)

# take shared cells
multiome_rna=adata_rna[barcodes_inter,:]
multiome_atac=adata_atac[barcodes_inter,:]

# concatenate
adata_paired=anndata.AnnData(
    X=scipy.sparse.hstack([multiome_rna.X,multiome_atac.X]).tocsr(), # concatenate sparse matrix and convert to Compressed Sparse Row format
    obs=multiome_rna.obs[['sangerID', 'combinedID', 'donor', 'region', 'age', 'sex', 'facility',
                           'cell_or_nuclei', 'modality', 'kit_10x', 'week', 'trimester', 'cycling',
                           'S_score', 'G2M_score', 'phase', 'fine_grain', 'mid_grain','coarse_grain','batch_key',
                          # ATAC QC

                          # RNA QC
                            'total_counts',
                     'total_counts_mt',
                     'total_counts_ribo',
                            'n_genes_by_counts',
                     'pct_counts_mt',
                     'pct_counts_ribo',
                            'log1p_n_genes_by_counts',
                     'log1p_total_counts',
                     'log1p_total_counts_mt',
                     'log1p_total_counts_ribo',
                     'scrublet_score',
                        'doublet_pval',
                        'doublet_bh_pval',]],
    var=pd.concat([multiome_rna.var[['modality','gene_name_scRNA-0','gene_id']], \
                   multiome_atac.var
                  ],axis=0)
)
adata_paired

AnnData object with n_obs × n_vars = 35705 × 544641
    obs: 'sangerID', 'combinedID', 'donor', 'region', 'age', 'sex', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'week', 'trimester', 'cycling', 'S_score', 'G2M_score', 'phase', 'fine_grain', 'mid_grain', 'coarse_grain', 'batch_key', 'total_counts', 'total_counts_mt', 'total_counts_ribo', 'n_genes_by_counts', 'pct_counts_mt', 'pct_counts_ribo', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_ribo', 'scrublet_score', 'doublet_pval', 'doublet_bh_pval'
    var: 'modality', 'gene_name_scRNA-0', 'gene_id', 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'distToGeneStart', 'nearestGene', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'

In [15]:
# modify modality columns in .obs
adata_paired.obs['modality'] = 'Multiome'
adata_paired.obs['modality'].value_counts()

Multiome    35705
Name: modality, dtype: int64

In [16]:
# save
adata_paired.write('/nfs/team205/heart/anndata_objects/Foetal/vCM/MultiVI-pre_paired_foetal_vCMs_raw.h5ad')

In [17]:
adata_paired.obs.head()

,sangerID,combinedID,donor,region,age,sex,facility,cell_or_nuclei,modality,kit_10x,...,n_genes_by_counts,pct_counts_mt,pct_counts_ribo,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mt,log1p_total_counts_ribo,scrublet_score,doublet_pval,doublet_bh_pval
barcode,,,,,,,,,,,,,,,,,,,,,
BHF_F_Hea11933666_BHF_F_Hea11596619_TAAGTAGCAGCATGTC-1,BHF_F_Hea11933666,BHF_F_Hea11933666_BHF_F_Hea11596619,Hst33,whole sample,13W0D,female,Sanger,nuclei,Multiome,Multiome-v1,...,2178,0.453559,0.670479,7.686621,8.531490,3.178054,3.555348,0.107016,0.494493,0.913587
BHF_F_Hea11933669_BHF_F_Hea11596622_AATTAGCGTTAGCATG-1,BHF_F_Hea11933669,BHF_F_Hea11933669_BHF_F_Hea11596622,Hst33,whole sample,13W0D,female,Sanger,nuclei,Multiome,Multiome-v1,...,2076,1.362089,1.570185,7.638680,8.573007,4.290460,4.430817,0.126360,0.370379,0.881283
BHF_F_Hea11933669_BHF_F_Hea11596622_CTGTATTTCGGGACTC-1,BHF_F_Hea11933669,BHF_F_Hea11933669_BHF_F_Hea11596622,Hst33,whole sample,13W0D,female,Sanger,nuclei,Multiome,Multiome-v1,...,1587,2.312312,2.102102,7.370231,8.111028,4.356709,4.262680,0.131783,0.383525,0.881283
7089STDY13216926_BHF_F_Hea13242533_CAAGACAAGCGCCTAA-1,7089STDY13216926,7089STDY13216926_BHF_F_Hea13242533,Hst39,heart without node,15W0D,female,Sanger,nuclei,Multiome,Multiome-v1,...,745,0.565504,0.094251,6.614726,6.967909,1.945910,0.693147,0.082673,0.454484,0.885838
HCAHeartST13386009_HCAHeartST13303419_ATCCTTAGTGGATTGC-1,HCAHeartST13386009,HCAHeartST13386009_HCAHeartST13303419,Hst41,atria,20W0D,male,Sanger,nuclei,Multiome,Multiome-v1,...,2299,2.856576,2.439992,7.740664,8.525558,4.976734,4.820282,0.086068,0.219175,0.733829


In [18]:
adata_paired.obs['batch_key'].value_counts()

Hst33_Nuclei_Multiome-v1    17373
Hst41_Nuclei_Multiome-v1     9591
Hst36_Nuclei_Multiome-v1     2152
Hst39_Nuclei_Multiome-v1     1932
C87_Nuclei_Multiome-v1       1440
Hst40_Nuclei_Multiome-v1     1429
C104_Nuclei_Multiome-v1       720
C98_Nuclei_Multiome-v1        421
C92_Nuclei_Multiome-v1        322
C85_Nuclei_Multiome-v1        280
C82_Nuclei_Multiome-v1         45
Name: batch_key, dtype: int64

# Prepare RNA data

In [19]:
# select single-cell data (3p and 5p)
adata_scrna = adata_rna[adata_rna.obs['kit_10x'].isin(['3prime-v3','5prime-v2'])]
adata_scrna.obs['kit_10x'].value_counts()

5prime-v2    4788
3prime-v3    4032
Name: kit_10x, dtype: int64

In [20]:
adata_scrna.X.data[:10]

array([6., 1., 3., 1., 1., 1., 8., 3., 1., 1.], dtype=float32)

In [21]:
# recreate anndata
adata_scrna=anndata.AnnData(
    X=adata_scrna.X, # concatenate sparse matrix and convert to Compressed Sparse Row format
    obs=adata_scrna.obs[['sangerID', 'combinedID', 'donor', 'region', 'age', 'sex', 'facility',
                           'cell_or_nuclei', 'modality', 'kit_10x', 'week', 'trimester', 'cycling',
                           'S_score', 'G2M_score', 'phase', 'fine_grain', 'mid_grain','coarse_grain','batch_key',
                          # ATAC QC

                          # RNA QC
                            'total_counts',
                     'total_counts_mt',
                     'total_counts_ribo',
                            'n_genes_by_counts',
                     'pct_counts_mt',
                     'pct_counts_ribo',
                            'log1p_n_genes_by_counts',
                     'log1p_total_counts',
                     'log1p_total_counts_mt',
                     'log1p_total_counts_ribo',
                     'scrublet_score',
                        'doublet_pval',
                        'doublet_bh_pval',]],
    var=adata_scrna.var[['modality','gene_name_scRNA-0','gene_id']]
)
adata_scrna

AnnData object with n_obs × n_vars = 8820 × 36601
    obs: 'sangerID', 'combinedID', 'donor', 'region', 'age', 'sex', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'week', 'trimester', 'cycling', 'S_score', 'G2M_score', 'phase', 'fine_grain', 'mid_grain', 'coarse_grain', 'batch_key', 'total_counts', 'total_counts_mt', 'total_counts_ribo', 'n_genes_by_counts', 'pct_counts_mt', 'pct_counts_ribo', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_ribo', 'scrublet_score', 'doublet_pval', 'doublet_bh_pval'
    var: 'modality', 'gene_name_scRNA-0', 'gene_id'

In [22]:
# modify modality columns in .obs
adata_scrna.obs['modality'] = 'scRNA'
adata_scrna.obs['modality'].value_counts()

scRNA    8820
Name: modality, dtype: int64

In [23]:
adata_scrna.var

,modality,gene_name_scRNA-0,gene_id
gene_name_multiome-1,,,
MIR1302-2HG,Gene Expression,MIR1302-2HG,ENSG00000243485
FAM138A,Gene Expression,FAM138A,ENSG00000237613
OR4F5,Gene Expression,OR4F5,ENSG00000186092
AL627309.1,Gene Expression,AL627309.1,ENSG00000238009
AL627309.3,Gene Expression,AL627309.3,ENSG00000239945
...,...,...,...
AC141272.1,Gene Expression,AC141272.1,ENSG00000277836
AC023491.2,Gene Expression,AC023491.2,ENSG00000278633
AC007325.1,Gene Expression,AC007325.1,ENSG00000276017


In [24]:
# save
adata_scrna.write('/nfs/team205/heart/anndata_objects/Foetal/vCM/MultiVI-pre_scRNA_foetal_vCMs_raw.h5ad')